### Imports

In [19]:
import pandas as pd
import numpy as np

### Data anlysis

In [3]:
data = pd.read_csv('data.csv')
data.head()

C:\Users\Arielle\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (14,28,37,38) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


IndexRow     Id                                           Logo Url  \
0    4,568  78752                                                NaN   
1    4,569  78752                                                NaN   
2    4,562  78749  https://cityoflearning-uploads.s3.amazonaws.co...   
3    4,563  78749  https://cityoflearning-uploads.s3.amazonaws.co...   
4    4,557  78746  https://cityoflearning-uploads.s3.amazonaws.co...   

                                  Program Name  \
0  BBBS Community Based Mentoring (Year Round)   
1  BBBS Community Based Mentoring (Year Round)   
2        Youth Mentoring and Academic Supports   
3        Youth Mentoring and Academic Supports   
4                Refugee Youth Career Pathways   

                                         Description  \
0  Community Based Mentoring is the traditional B...   
1  Community Based Mentoring is the traditional B...   
2  Young Men's Educational Network provides mento...   
3  Young Men's Educational Network provides mento...   
4  Refugee Youth Career Pathways (RYCP) provides ...   

                                Org Name     Category Name  Capacity  Min Age  \
0               Big Brothers Big Sisters  Academic Support    2000.0        7   
1               Big Brothers Big Sisters    Work + Career     2000.0        7   
2  Youth Mentoring and Academic Supports  Academic Support     100.0       12   
3  Youth Mentoring and Academic Supports      Music & Art.     100.0       12   
4                             RefugeeOne   Managing Money.      15.0       16   

   Max Age  ...             Contact Email Contact Phone Program Price  \
0       15  ...  recruitment@bbbschgo.org           NaN          Free   
1       15  ...  recruitment@bbbschgo.org           NaN          Free   
2       18  ...  wchatman@ymenchicago.com           NaN          Free   
3       18  ...  wchatman@ymenchicago.com           NaN          Free   
4       24  ...                       NaN           NaN          Free   

  Geographic Cluster Name Program Pays Participants Program Has Scholarships  \
0          NEAR WEST SIDE                  Not Paid                      YES   
1          NEAR WEST SIDE                  Not Paid                      YES   
2          NORTH LAWNDALE                  Not Paid                       NO   
3          NORTH LAWNDALE                  Not Paid                       NO   
4                     NaN                  Not Paid                      YES   

  Program Provides Transportation   Latitude  Longitude  \
0                             YES  41.886002 -87.642403   
1                             YES  41.886002 -87.642403   
2                              NO  41.864799 -87.724800   
3                              NO  41.864799 -87.724800   
4                             YES        NaN        NaN   

  Program Provides Free Food  
0                      False  
1                      False  
2                       True  
3                       True  
4                      False  

[5 rows x 56 columns]

In [4]:
data.columns

Index(['IndexRow', 'Id', 'Logo Url', 'Program Name', 'Description', 'Org Name',
       'Category Name', 'Capacity', 'Min Age', 'Max Age', 'Meeting Type',
       'Address', 'City', 'State', 'Zipcode', 'Online Address', 'Program Url',
       'Registration Url', 'Registration Open', 'Registration Deadline',
       'Start Date', 'End Date', 'Start Time', 'End Time', 'Scheduled Mon',
       'Scheduled Tues', 'Scheduled Fri', 'Scheduled Sat', 'Scheduled Sun',
       'Scheduled Thurs', 'Scheduled Wed', 'Fri End Time', 'Fri Start Time',
       'Mon End Time', 'Mon Start Time', 'Sat End Time', 'Sat Start Time',
       'Sun End Time', 'Sun Start Time', 'Thurs End Time', 'Thurs Start Time',
       'Tues End Time', 'Tues Start Time', 'Wed End Time', 'Wed Start Time',
       'Contact Name', 'Contact Email', 'Contact Phone', 'Program Price',
       'Geographic Cluster Name', 'Program Pays Participants',
       'Program Has Scholarships', 'Program Provides Transportation',
       'Latitude', 'Longitu

In [16]:
data['Scheduled Mon'].value_counts()

True    161
Name: Scheduled Mon, dtype: int64

Possible variables:
- weekday y/n
- during school hours y/n
- during work hours y/n
- age range width: kids, tweens, teens, young adults
- registration availability length 
- (if data possible from URL) registered on app or website 

In [8]:
data.iloc[1,1]

78752

In [10]:
type(data.iloc[1,18])

str

In [13]:
#cleaning to engineer time related variables
data['start_date'] = pd.to_datetime(data['Start Date'])
data['end_date'] = pd.to_datetime(data['End Date'])
data['registration_open'] = pd.to_datetime(data['Registration Open'])
data['registration_close'] = pd.to_datetime(data['Registration Deadline'])
data['start_time'] = pd.to_datetime(data['Start Time'])
data['end_time'] = pd.to_datetime(data['End Time'])

In [17]:
#creating temporal variables 
data['program_daily_len'] = data['end_time'] - data['start_time']
data['registration_len'] = data['registration_close'] - data['registration_open']
data['program_len'] = data['end_date']-data['start_date']
data['registration_to_start_len'] = data['start_date'] -data['registration_close']



In [29]:
data['during_weekend'] = np.where((data['Scheduled Sat'] == True) | (data['Scheduled Sun']==True), 1, 0)


data['during_weekday'] = np.where(((data['Scheduled Mon'] ==True) | 
                                        (data['Scheduled Tues'] == True) | 
                                        (data['Scheduled Fri'] == True) | 
                                        (data['Scheduled Thurs']==True) |
                                        (data['Scheduled Wed']==True)),1, 0)



data['full_week'] = np.where((data['during_weekday'] == 1) & (data['during_weekend'] == 1), 1, 0)




In [46]:
#data['interrupts_workday'] = np.where((data['during_weekday'] == 1) & 
#                                      (((data['start_time'].hour> 9) & (data['start_time'].hour < 17)) | ((data['end_time'].hour > 9) & (data['end_time'].hour < 17))),
#                                     1,0)
                             

#data['interrupts_workday'] = data.apply(lambda x: if )





data['interrupts_workday'] = 0

for i in range(data.shape[0]):
    if data.loc[i,'during_weekday'] == 1:
        if (((data.loc[i,'start_time'].hour> 9) & (data.loc[i,'start_time'].hour < 17)) | 
            ((data.loc[i,'end_time'].hour > 9) & (data.loc[i,'end_time'].hour < 17))):
            data.loc[i,'interrupts_workday'] = 1
        


In [48]:
data['age_range'] = data['Max Age'] - data['Min Age']

In [53]:
#kids <= 10
#tweens 11,12,13
#teens 14-19
#young adults >=20

data['supports_kids'] = np.where((data['Min Age'] < 10), 1, 0)
data['supports_tweens'] = np.where((data['Min Age'] <13) & (data['Max Age']>11), 1, 0)
data['supports_young_adults'] = np.where((data['Max Age'] > 20),1,0)
data['supports_teens'] = np.where((data['Min Age'] < 20) & (data['Max Age'] > 13),1,0)

In [54]:

data['during_school_hours'] = 0

for i in range(data.shape[0]):
    if data.loc[i,'during_weekday'] == 1:
        if (((data.loc[i,'start_time'].hour> 8) & (data.loc[i,'start_time'].hour < 15)) | 
            ((data.loc[i,'end_time'].hour > 8) & (data.loc[i,'end_time'].hour < 15))):
            data.loc[i,'during_school_hours'] = 1
        